## **1. Specify BIDS directory**

In [1]:
from bids import BIDSLayout

bids_dataset_path = '/data/nimlab/USERS/joseph/collaborators/calvin/test_bids_dataset'

layout = BIDSLayout(bids_dataset_path, validate=False, absolute_paths=True)
print("BIDS dataset identified at", layout.root, "\nInfo:")
layout.__repr__()

BIDS dataset identified at /data/nimlab/USERS/joseph/collaborators/calvin/test_bids_dataset 
Info:


'BIDS Layout: ...ators/calvin/test_bids_dataset | Subjects: 3 | Sessions: 0 | Runs: 0'

## **2. Loop through subjects and collect T1w images**

In [2]:
import os

subs = layout.get_subjects()
sys_commands = ["source activate /data/nimlab/software/miniforge_2024-08-31/packages/envs/easyreg_standalone_env;"]
for i, sub in enumerate(subs):
    t1_files_for_sub = layout.get(
        subject=sub,
        suffix='T1w',
        return_type='filename'
    )
    first_t1_file = t1_files_for_sub[0] if len(t1_files_for_sub) > 0 else None
    if first_t1_file is None:
        print(f"Could not find T1w file for subject {sub} ({i+1}/{len(subs)})")
        continue
    print(f"Found {os.path.basename(first_t1_file)} from subject {sub} ({i+1}/{len(subs)})")
    sys_commands.append(f"segment-csf-gm-wm {first_t1_file};")

Found 0002_T1w.nii from subject mytest1 (1/3)
Found 33061_T1w.nii.gz from subject mytest2 (2/3)
Found 33821_T1w.nii.gz from subject mytest3 (3/3)


## **3. Submit to slurm job scheduler**

In [3]:
cores = 11
memory = 16000
job_time = "23:59:00"
job_name = "segment_csf_gm_wm"
partition = "nimlab,normal,long,bigmem"
nodes = 1
user_email = "jiturner@bwh.harvard.edu"
output_dir = ""

script_lines = ['#!/bin/bash']
script_lines.extend([
    f'#SBATCH -J {job_name}',
    f'#SBATCH -p {partition}',
    f'#SBATCH --nodes {nodes}',
    f'#SBATCH --ntasks {nodes}',
    f'#SBATCH -c {cores}',
    f'#SBATCH --mem {memory}',
    f'#SBATCH -t {job_time}',
    f'#SBATCH --mail-user={user_email}',
    '#SBATCH --mail-type=END'
])

if not output_dir:
    output_dir = os.path.join(os.getcwd())
if output_dir:
    script_lines.extend([
        f'#SBATCH -o {output_dir}/slurm.%N.%j.out',
        f'#SBATCH -e {output_dir}/slurm.%N.%j.err'
    ])

script_lines.append('')
script_lines.extend(sys_commands)
result = '\n'.join(script_lines)

# Write the script to a file
script_path = os.path.join(output_dir, f'{job_name}.sh')
with open(script_path, 'w') as f:
    f.write(result)

print(f"Script written to {script_path}")
print(f"Please ssh to eristwo and submit the following command:")
print(f"sbatch {script_path}")

Script written to /data/nimlab/USERS/joseph/collaborators/calvin/segment_csf_gm_wm.sh
Please ssh to eristwo and submit the following command:
sbatch /data/nimlab/USERS/joseph/collaborators/calvin/segment_csf_gm_wm.sh
